In [22]:
# eeg analysis libraries
import mne
from scipy import io
from neurodsp import spectral
from neurodsp.plts.spectral import plot_spectral_hist

import pkg_resources

import ssvepy

from ssvepy import Ssvep, load_ssvep
from ssvepy import frequencymaths

__version__ = pkg_resources.get_distribution("ssvepy").version


# import plotting libraries
from matplotlib import pyplot as plt  
import seaborn as sns
from plotnine import *

# numerical libraries
import numpy as np
import pandas as pd
from pandas.plotting import table 
import imgkit

# import tqdm for a progress bar:
try:
    from tqdm import tqdm_notebook as tqdm
except:
    from tqdm import tqdm
    
# import file path operators
from pathlib import Path
import os
import six

# avoid MNE being too verbose
mne.set_log_level('ERROR')

In [23]:
datafolder = Path('/Data2/EEG/Data/EEG_SSVEP_Faces_OB_renamed')
datafiles = list(datafolder.glob(pattern='*Categorisation.bdf'))
group = [f.name[3:6] for f in datafiles]
pids = [f.name[7:11] for f in datafiles]
#print(group)
files = os.listdir(datafolder)
for name in datafiles:
    print(name)

/Data2/EEG/Data/EEG_SSVEP_Faces_OB_renamed/EG-CTR-1022-Face_Categorisation.bdf
/Data2/EEG/Data/EEG_SSVEP_Faces_OB_renamed/EG-CTR-1012-Face_Categorisation.bdf
/Data2/EEG/Data/EEG_SSVEP_Faces_OB_renamed/EG-CTR-1002-Face_Categorisation.bdf
/Data2/EEG/Data/EEG_SSVEP_Faces_OB_renamed/EG-CTR-1007-Face_Categorisation.bdf
/Data2/EEG/Data/EEG_SSVEP_Faces_OB_renamed/EG-CTR-1025-Face_Categorisation.bdf
/Data2/EEG/Data/EEG_SSVEP_Faces_OB_renamed/EG-CTR-1011-Face_Categorisation.bdf
/Data2/EEG/Data/EEG_SSVEP_Faces_OB_renamed/EG-CTR-1018-Face_Categorisation.bdf
/Data2/EEG/Data/EEG_SSVEP_Faces_OB_renamed/EG-CTR-1035-Face_Categorisation.bdf
/Data2/EEG/Data/EEG_SSVEP_Faces_OB_renamed/EG-CTR-1023-Face_Categorisation.bdf
/Data2/EEG/Data/EEG_SSVEP_Faces_OB_renamed/EG-CTR-1026-Face_Categorisation.bdf
/Data2/EEG/Data/EEG_SSVEP_Faces_OB_renamed/EG-CTR-1033-Face_Categorisation.bdf
/Data2/EEG/Data/EEG_SSVEP_Faces_OB_renamed/EG-CTR-1021-Face_Categorisation.bdf
/Data2/EEG/Data/EEG_SSVEP_Faces_OB_renamed/EG-CTR-10

In [24]:
datadicts = []
electrodes_snr = []
ssveps = []
snrs_faces = []
snrs_ob = []
Report_pp = []

for file in tqdm(datafiles):
    raw = mne.io.read_raw_edf(
        file, montage=mne.channels.read_montage('biosemi64'),preload=True,
        eog=[f'EXG{n}' for n in range(1, 9)]
    )
    raw.info['subject_info'] = {
        'pid': file.name[7:11],
        'group': file.name[3:6],
        'filename' :file.name
    }
    
    events = mne.find_events(raw, stim_channel='STI 014')
    events = events[events[:, 2] < 255, :]
    
    raw.info['events'] = events
    #print(events[:5])
    
    #Apply Frequency Filter
    raw.filter(0.5, 15)
    
    #Preprocessing, including demean and detrend.
    #detrend=0 - constant detrending, detrend=1 linear detrending
    epochs = mne.Epochs(
        raw, events,
        tmin=0, tmax=4, preload=True,
        baseline=(None, None), detrend=0,
    ).set_eeg_reference().load_data().apply_proj()
    
    evoked = epochs.average()

    ssvep = ssvepy.Ssvep(epochs, [1.2, 6], compute_tfr=True)
    
    # Define electrodes: Occipital & All Electrodes
    o_elecs = ['Oz', 'O1', 'O2', 'Iz']
    occipital_elecs = mne.pick_channels(raw.ch_names, o_elecs)
    
    elecs = ['Fp1','AF7','AF3','F1','F3','F5','F7','FT7','FC5',
             'FC3','FC1','C1','C3','C5','T7','TP7','CP5','CP3',
             'CP1','P1','P3','P5','P7','P9','PO7','PO3','O1',
             'Iz','Oz','POz','Pz','CPz','Fpz','Fp2','AF8','AF4',
             'AFz','Fz','F2','F4','F6','F8','FT8','FC6','FC4',
             'FC2','FCz','Cz','C2','C4','C6','T8','TP8','CP6',
             'CP4','CP2','P2','P4','P6','P8','P10','PO8','PO4','O2',]
    
    all_elecs = mne.pick_channels(raw.ch_names, elecs)
    
    file_id = raw.info['subject_info']['filename']
    
    # Collect snrs for face and object frequency for all electrodes. 
    key = (len(elecs))
    snrs_faces = []
    snrs_ob = []
    difference = []
    
    # use i as counter for electrodes
    for i in range(key): 
        snrs_faces.insert(i,  ssvep.stimulation.snr.loc[:, elecs[i], 6].values.mean())
        snrs_ob.insert(i, ssvep.stimulation.snr.loc[:, elecs[i], 1.2].values.mean())
        dif = ((snrs_faces[i] - snrs_ob[i]) / snrs_faces[i])
        difference.insert(i, dif)
        
    # Make Dataframe of the snrs for faces, for objects, and for the difference between these values per electrode. 
        
    d = {'Electrodes': elecs, 'SNR Face': snrs_faces, 'SNR Object': snrs_ob, 'difference': difference,}
    electrodes_snr = pd.DataFrame(data=d, index=elecs) 
    elecs_snr_sorted = electrodes_snr.sort_values(by=["SNR Face"], ascending = False)
    df = pd.DataFrame(elecs_snr_sorted).head(5)
    top_electrodes = df.set_index("Electrodes", drop = False)
        
    # Create dictionary for each file with results and collect them in datadicts. 
    Report_pp = {'pid': raw.info['subject_info']['pid'],
         'group': raw.info['subject_info']['group'],
         'filename': raw.info['subject_info']['filename'],
         'ssvep': ssvep,
         'Top Electrodes': top_electrodes,
        
    # index using the electrode strings, and the raw frequency value:
         
        'Objects': ssvep.stimulation.snr.loc[:, o_elecs , 1.2].mean().values,
        'Faces': ssvep.stimulation.snr.loc[:, o_elecs , 6].mean().values,}
    
    datadicts.append(Report_pp) 
    pid = Report_pp['pid']
    path = ('results_ssvep/all_electrodes/%s' %(file_id))
    if not os.path.exists(path):
        os.makedirs(path)
        
    # Save all electrodes to file 
    pd.DataFrame(electrodes_snr).to_csv('%s/all_electrodes_snrs_%s.csv' %(path, file_id));

    
    
# save everything to a CSV file ##
path = ('results_ssvep/occipital_electrodes')
if not os.path.exists(path):
    os.makedirs(path)
pd.DataFrame(datadicts).to_csv('%s/occipital_electrodes_snrs.csv' %(path));

In [25]:
#make report for each participant

file_length = len(datadicts)

#define paths
for file in range(file_length):
    pid = datadicts[file]['pid']
    fid = datadicts[file]['filename']
    sid = datadicts[file]['ssvep']
    path = ('results_ssvep/%s' %(pid))

    if not os.path.exists(path):
        os.makedirs(path)

    power_mean = sid.stimulation.power.mean()
    sid.harmonic.frequencies
    sid.harmonic.power.shape
    sid.stimulation.tfr.shape
    
    # Make plots   
    sid.topoplot_psd(flims='harmonic', figsize=(12, 10))
    plt.savefig('%s/topoplot_psd_%s.png' %(path, fid))
    plt.clf()

    sid.plot_psd()
    plt.savefig('%s/plot_psd_%s.png' %(path, fid))
    plt.clf()

    sid.plot_snr()
    plt.savefig('%s/plot_snr_%s.png' %(path, fid))
    plt.clf()

    sid.topoplot_snr(flims='harmonic', figsize=(12, 10))
    plt.savefig('%s/topoplot_snr_%s.png' %(path, fid))
    plt.clf()

    # make table

    df = datadicts[file]['Top Electrodes']
    def render_mpl_table(data, col_width=3.0, row_height=0.625, font_size=14, rowLabels= df.index,
                         header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='w',
                         bbox=[0, 0, 1, 1], header_columns=0,
                         ax=None, **kwargs):
        if ax is None:
            size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
            fig, ax = plt.subplots(figsize=size)
            ax.axis('off')

        mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, rowLabels=data.index, **kwargs)

        mpl_table.auto_set_font_size(False)
        mpl_table.set_fontsize(font_size)

        for k, cell in six.iteritems(mpl_table._cells):
            cell.set_edgecolor(edge_color)
            if k[0] == 0 or k[1] < header_columns:
                cell.set_text_props(weight='bold', color='w')
                cell.set_facecolor(header_color)
            else:
                cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
        return ax


    render_mpl_table(df, col_width=4.0)
    plt.savefig('%s/Top_Electrodes_%s.png' %(path, fid))
    plt.clf()

    from fpdf import FPDF
    pdf = FPDF()
    # figlist = [, '%s/topoplot_psd_%s.png' %(path, fid),  ]
    # imagelist is the list with all image filenames

    #print report
    titlelist = ['SNR plot', 'PSD plot', 'SNR topoplot', 'PSD topoplot', 'Top Electrodes']
    t = 0
    pdf.add_page()
    
    #add plots
    pdf.image('%s/plot_snr_%s.png' %(path, fid), 0 ,20, 0, 90 )
    pdf.image('%s/plot_psd_%s.png' %(path, fid), 100 ,20, 0, 90 )
    pdf.image('%s/topoplot_snr_%s.png' %(path, fid), 0 ,120, 0, 100 )
    pdf.image('%s/topoplot_psd_%s.png' %(path, fid), 100 ,120, 0, 100 )
    pdf.image('%s/Top_Electrodes_%s.png' %(path, fid), -10 , 230 , 200 ,0)
    
    # add titles
    pdf.set_font('courier')
    pdf.write(5, titlelist[0])
    for space in range(6):
        pdf.write(5, '     ')
    pdf.write(5, titlelist[1])
    for space in range(20):
        pdf.write(5, '\n')
    pdf.write(5, titlelist[2]) 
    for space in range(5):
        pdf.write(5, '     ')
    pdf.write(5, titlelist[3])
    for space in range(22):
        pdf.write(5, '\n')
    pdf.write(5, titlelist[4])

    #Save pdf
    pdf.output("%s/report_%s.pdf" %(path, fid), "F") 

/home/richard/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/home/richard/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/home/richard/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicit

/home/richard/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/home/richard/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/home/richard/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicit

/home/richard/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/home/richard/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/home/richard/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicit

<Figure size 864x720 with 0 Axes>

<Figure size 864x720 with 0 Axes>

...

<Figure size 864x720 with 0 Axes>

<Figure size 1152x270 with 0 Axes>

<Figure size 864x720 with 0 Axes>

<Figure size 864x720 with 0 Axes>

<Figure size 1152x270 with 0 Axes>